# Описание проекта

### Друзык Роман Богданович

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# 1. Загрузка и подготовка данных

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

*Загрузим данные и посмотрим на их структуру*

In [2]:
field_0 = pd.read_csv('/datasets/geo_data_0.csv')
field_1 = pd.read_csv('/datasets/geo_data_1.csv')
field_2 = pd.read_csv('/datasets/geo_data_2.csv')

***

*Исследуем данные с первого региона*

In [3]:
field_0.head(3)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647


In [4]:
field_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


*Посмотрим на наличие пропусков*

In [5]:
field_0.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

*Посмотрим на количество дубликатов в данных*

In [6]:
field_0.id.duplicated().sum()

10

*Данные дубли не будут влиять на работу модели, так как столбец id не используется при дальнейшей работе*

***
***

*Исследуем данные из второго региона*

In [7]:
field_1.head(3)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305


In [8]:
field_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


*Посмотрим на наличие пропусков*

In [9]:
field_1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

*Посмотрим на количество дубликатов в данных*

In [10]:
field_1.id.duplicated().sum()

4

*Данные дубли не будут влиять на работу модели, так как столбец id не используется при дальнейшей работе*

***
***

*Исследуем данные из третьего региона*

In [11]:
field_2.head(3)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910


In [12]:
field_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


*Посмотрим на наличие пропусков*

In [13]:
field_2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

*Посмотрим на количество дубликатов в данных*

In [14]:
field_2.id.duplicated().sum()

4

*Данные дубли не будут влиять на работу модели, так как столбец id не используется при дальнейшей работе*

***

### Вывод

Исходные данные достались довольно чистые, без пропусков, с корректными типами данных в признаках. Единственное что мы нашли - дубли id.

***
***

# 2. Обучение и проверка модели

*Обучим три модели линейной регрессии для каждого месторождения. Каждый набор данных разобъем на обучающую и тестовую выборки в пропорции 75:25.*

*Подготовим наши наборы данных, разбив их на признаки и целевую переменную, отбросив id, так как мы не планируем привязываться к конкретной скважине, а хотим оценить рентабельность месторождения в среднем*

In [15]:
'''
Функция, которая принимает датасет и модель, далее разбивает датасет на целевую перменную и признаки, обучается на модели
и возвращает предсказания и RMSE
'''
def model_prediction(data):
    model = LinearRegression()
    X = data[['f0', 'f1', 'f2']]
    y = data['product']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)
    predicted = pd.Series(model.fit(X_train, y_train).predict(X_test), index=y_test.index)
    score = mean_squared_error(y_test, predicted) ** 0.5
    return y_test, predicted, score

In [16]:
#Получим предсказания и RMSE по трем месторождениям
target_field_0, pred_field_0, rmse_field_0 = model_prediction(field_0)
target_field_1, pred_field_1, rmse_field_1 = model_prediction(field_1)
target_field_2, pred_field_2, rmse_field_2 = model_prediction(field_2)

In [17]:
#Для удобного представления упакуем наши результаты в DataFrame
results = pd.DataFrame(data=[[pred_field_0.mean(), rmse_field_0], 
                             [pred_field_1.mean(), rmse_field_1], 
                             [pred_field_2.mean(), rmse_field_2]],
                       index=['Месторождение fields_0', 'Месторождение fields_1','Месторождение fields_2'], 
                       columns=['Предсказанный средний запас сырья', 'RMSE модели'])
results

,Предсказанный средний запас сырья,RMSE модели
Месторождение fields_0,92.398800,37.756600
Месторождение fields_1,68.712878,0.890280
Месторождение fields_2,94.771024,40.145872


### Вывод

*Мы провели обучение модели по трем месторождениям и получили предсказания на тестовых данных. По полученным данным мы видим что средние запасы сырья на месторождениях с индексом 0 и 2 примерно соизмеримы друг с другом, но точность RMSE модели по этим данным - оставляет желать лучшего. В то же время запасы на месторождении с индексом 1 гораздо меньше предыдущих территорий, но точность модели практически максимальная.*

***
***

# 3. Подготовка к расчёту прибыли

*Сохраним все необходимые экономические константы для расчета прибыли/убытков месторождений*

In [18]:
#Максимальное количество разведываемых скважин
MAXIMUM_NUMBERS_OF_WELLS = 500

#Максимальный бюджет на разработку
BUDGET = 10 * 10**9

#Стоимость разработки одной скважины
BUDGET_TO_ONE_WELL = 50 * 10**6

#Стоимость тысячи баррелей нефти
PRICE_TO_THOUSAND_BARREL = 4500 * 1000

#Максимальное количество скважин, которое возможно разработать в рамках бюджета
MAXIMUM_EXPLORED_WELL = BUDGET / BUDGET_TO_ONE_WELL

*Расчитаем минимальный средний объем сырья в месторождениях региона, достаточный для его разработки, по следующей формуле:*

*Минимальный средний объем сырья умножим на стоимость одного барреля нефти и умножим на максимальное количество скважин, которое возможно разработать в рамках бюджета, данная сумма должна быть равна максимальному бюджету на разработку.*

*Из этого простого уравнения мы можем найти минимальный средний обзем сырья, как максимальный бюджет на разработку, деоенный на произведение максимального количества скважин на стоимость одного барреля*

In [19]:
MIN_MEAN_VOLUME_MINING = BUDGET / (MAXIMUM_EXPLORED_WELL * PRICE_TO_THOUSAND_BARREL)
print('Минимальный средний объем сырья в месторождениях региона: {:.2f} тыс. баррелей'.format(MIN_MEAN_VOLUME_MINING))

Минимальный средний объем сырья в месторождениях региона: 11.11 тыс. баррелей


In [20]:
'''
Функция, которая принимает массив с предсказанным объемом нефти и количеством скважин, а возвращает упорядоченный по
убыванию массив предполагаемой прибыли (в млн. рублей) со скважины, при 100% выработке нефти.
'''
def revenue_mining(predicted, count):
    return (PRICE_TO_THOUSAND_BARREL * predicted[:count] - BUDGET_TO_ONE_WELL)

### Вывод

*Предварительно, по нашим расчетам, месторождение, в котором средний запас нефти в скважине больше 11 тыс. баррелей должно быть потенциально прибыльно*

***
***

# 4. Расчёт прибыли и рисков 

*Мы предполагаем, что компания может взять для исследования 500 скважин, а физически начать добычу на 200 скважинах из 500 исследованных, с самыми высокими оценками запасов, что становится ясным из предполагаемого бюджета на разработку и стоимости разработки одной скважины.*

*Риск убытков определим как отношение убыточных скважин к общему количеству разрабатываемых скважин*

In [27]:
def bootstrap_sample(predicted, n_well=500, n_explored=200):
    
    state = np.random.RandomState(45)    
    total_values = []
    minus_values = []
    proba_minus_values = []
    
    for i in range(1000):
        #Выбираем n_well произвольных скважин 
        prediction_sample = predicted.sample(n=n_well, replace=True, random_state=state)
        
        #Считаем прибыль n_explored скважинам из n_well выбранных
        total_revenue = revenue_mining(prediction_sample, n_explored)
        
        #выбираем из выборки скважины с отрицательной доходностью
        minus_revenue = total_revenue[total_revenue<=0]
        
        #Считаем вероятность получить убыток
        proba_minus_revenue = len(minus_revenue) / len(total_revenue)
        
        #Сохраним прибыль
        total_values.append(total_revenue.sum())
        
        #Coхраним убытки
        minus_values.append(minus_revenue.sum())
        
        #Сохраним вероятности убытков
        proba_minus_values.append(proba_minus_revenue)
        

    #массивы приведем к Series значений в млн. рублей
    total_values = pd.Series(total_values) / 1000000
    minus_values = pd.Series(minus_values) / 1000000
    proba_minus_values = pd.Series(proba_minus_values)
    
    #Cредняя прибыль
    total_mean = total_values.mean() 
    #Нижняя граница доверительного интервала средней прибыли
    lower_total_mean = round(total_values.quantile(.025), 2)
    #Верхняя граница доверительного интервала средней прибыли
    higher_total_mean = round(total_values.quantile(.975), 2)
    
    #Cредний убыток
    minus_values_mean = minus_values.mean()
    #Нижняя граница доверительного интервала среднего убытка
    lower_minus_values = round(minus_values.quantile(.025), 2)
    #Верхняя граница доверительного интервала среднего убытка
    higher_minus_values = round(minus_values.quantile(.975), 2)
    
    #Cредняя вероятность получить убыток
    proba_minus_values_mean = proba_minus_values.mean()
    #Нижняя граница доверительного интервала вероятности убытка
    lower_proba_minus_values = round(proba_minus_values.quantile(.025), 2)
    #Верхняя граница доверительного интервала вероятности убытка
    higher_proba_minus_values = round(proba_minus_values.quantile(.975), 2)

    return pd.DataFrame(data=[[total_mean], [(lower_total_mean, higher_total_mean)], 
                              [minus_values_mean], [(lower_minus_values, higher_minus_values)], 
                              [proba_minus_values_mean], [(lower_proba_minus_values, higher_proba_minus_values)]], 
                        index=['Средняя прибыль', 'Доверительный интервал средней прибыли', 
                               'Cредний убыток', 'Доверительный интервал среднего убытка', 
                               'Cредняя вероятность убыта', 'Доверительный интервал вероятности убытка'])

*Применим нашу функцию к трем данным о месторождениях*

In [28]:
data_field_0 = bootstrap_sample(pred_field_0)
data_field_0.columns = ['Месторождение fields_0']
data_field_1 = bootstrap_sample(pred_field_1)
data_field_1.columns = ['Месторождение fields_1']
data_field_2 = bootstrap_sample(pred_field_2)
data_field_2.columns = ['Месторождение fields_2']

In [23]:
#Объеденим получившиеся результаты в едунную сумарную таблицу
results = results.merge(data_field_2
                        .merge(data_field_1
                               .merge(data_field_0, 
                                      left_index=True, 
                                      right_index=True), 
                               left_index=True, 
                               right_index=True).T, 
                        left_index=True, 
                        right_index=True).T

*Посмотрим на результат*

In [24]:
results

,Месторождение fields_0,Месторождение fields_1,Месторождение fields_2
Предсказанный средний запас сырья,92.3988,68.7129,94.771
RMSE модели,37.7566,0.89028,40.1459
Средняя прибыль,73144,51723.8,75330.2
Доверительный интервал средней прибыли,"(70128.41, 76176.73)","(45838.73, 57253.66)","(72762.71, 77916.31)"
Cредний убыток,-1.63835,-1435.1,0
Доверительный интервал среднего убытка,"(-11.73, 0.0)","(-1877.82, -988.74)","(0.0, 0.0)"
Cредняя вероятность убыта,0.00023,0.167395,0
Доверительный интервал вероятности убытка,"(0.0, 0.0)","(0.12, 0.22)","(0.0, 0.0)"


### Вывод

 - По полученым результатам, мы можем сделать вывод, что рентабельность добычи будет исключительно положительной, только на месторождении field_2. 
 - Месторождение field_1 не рекомендовано к разработке, так как средняя вероятность получения убытка составляет 16%.  
 - Месторождение fields_0 так же имеет очень низкую вероятность получения убытка, разработка допустима.